## Background:

- Dataset Information

The NHANES dataset was created to assess the health and nutritional status of adults and children in the United States. This dataset is funded by the Centers for Disease Control and Prevention (CDC), specifically through its National Center for Health Statistics (NCHS). The data is based on the survey respondents throughout the United States. Data was gathered through interviews, physical examinations, and laboratory tests. For this subset respondents 65 years old and older were labeled as “senior” and all individuals under 65 years old as “non-senior.”

### Task:

As a data analyst, your task is to prepare and analyse the data set using appropriate data preparation and apply the machine learning models for the specified tasks. Your analysis should aim to identify any relationships or trends in the data. Recognise suitable independent variables and target variable from the dataset to justify your classification/ prediction tasks. You can perform clustering on the dataset based on your analysis if applicable. You can generate synthetic data or use any additional data to support your machine learning models if required.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Dataset dictionary

- SEQN: ID, Respondent Sequence Number.
- age_group: Respondent's Age Group (senior (65 or older)/non-senior).
- RIDAGEYR: Respondent's Age.
- RIAGENDR: Respondent's Gender, a 1 represents Male and 2 represents Female.
- PAQ605: A 1 represents that the respondent takes part in weekly moderate or vigorous-intensity physical activity and a 2 represents that they do not.
- BMXBMI: Respondent's Body Mass Index.
- LBXGLU: Respondent's Blood Glucose after fasting.
- DIQ010: If the Respondent is diabetic.
- LBXGLT: Respondent's Oral.
- LBXIN: Respondent's Blood Insulin Levels.

In [3]:
df = pd.read_csv('NHANES_age_prediction.csv')
df.head(-5)

,SEQN,age_group,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,73564.0,Adult,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91
1,73568.0,Adult,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85
2,73576.0,Adult,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14
3,73577.0,Adult,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15
4,73580.0,Adult,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92
...,...,...,...,...,...,...,...,...,...,...
2268,83692.0,Senior,68.0,1.0,2.0,19.1,89.0,2.0,136.0,1.56
2269,83694.0,Adult,36.0,2.0,2.0,25.3,99.0,2.0,128.0,20.72
2270,83699.0,Adult,37.0,2.0,2.0,20.8,83.0,2.0,113.0,1.93
2271,83702.0,Senior,80.0,2.0,2.0,27.9,111.0,2.0,72.0,13.91


In [5]:
df.shape

(2278, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278 entries, 0 to 2277
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SEQN       2278 non-null   float64
 1   age_group  2278 non-null   object 
 2   RIDAGEYR   2278 non-null   float64
 3   RIAGENDR   2278 non-null   float64
 4   PAQ605     2278 non-null   float64
 5   BMXBMI     2278 non-null   float64
 6   LBXGLU     2278 non-null   float64
 7   DIQ010     2278 non-null   float64
 8   LBXGLT     2278 non-null   float64
 9   LBXIN      2278 non-null   float64
dtypes: float64(9), object(1)
memory usage: 178.1+ KB


In [7]:
df.describe()

,SEQN,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
count,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000
mean,78691.853819,41.795874,1.511414,1.822651,27.955180,99.553117,2.016242,114.978929,11.834794
std,2921.365151,20.156111,0.499979,0.398918,7.248962,17.889834,0.185556,47.061239,9.718812
min,73564.000000,12.000000,1.000000,1.000000,14.500000,63.000000,1.000000,40.000000,0.140000
25%,76171.750000,24.000000,1.000000,2.000000,22.800000,91.000000,2.000000,87.000000,5.860000
50%,78749.000000,41.000000,2.000000,2.000000,26.800000,97.000000,2.000000,105.000000,9.040000
75%,81213.500000,58.000000,2.000000,2.000000,31.200000,104.000000,2.000000,130.000000,14.440000
max,83727.000000,80.000000,2.000000,7.000000,70.100000,405.000000,3.000000,604.000000,102.290000


In [8]:
df.isnull().mean()*100 # Porcentage of missing data on each column.

SEQN         0.0
age_group    0.0
RIDAGEYR     0.0
RIAGENDR     0.0
PAQ605       0.0
BMXBMI       0.0
LBXGLU       0.0
DIQ010       0.0
LBXGLT       0.0
LBXIN        0.0
dtype: float64

In [9]:
df['age_group'].unique()

array(['Adult', 'Senior'], dtype=object)

In [11]:
df['senior'] = df['age_group'].replace(['Adult', 'Senior'], [0, 1])
df.head(-5)

,SEQN,age_group,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,senior
0,73564.0,Adult,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,0
1,73568.0,Adult,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,0
2,73576.0,Adult,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,0
3,73577.0,Adult,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15,0
4,73580.0,Adult,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,0
...,...,...,...,...,...,...,...,...,...,...,...
2268,83692.0,Senior,68.0,1.0,2.0,19.1,89.0,2.0,136.0,1.56,1
2269,83694.0,Adult,36.0,2.0,2.0,25.3,99.0,2.0,128.0,20.72,0
2270,83699.0,Adult,37.0,2.0,2.0,20.8,83.0,2.0,113.0,1.93,0
2271,83702.0,Senior,80.0,2.0,2.0,27.9,111.0,2.0,72.0,13.91,1


In [12]:
df.drop(columns='age_group', inplace=True)
df.head(-5)

,SEQN,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,senior
0,73564.0,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,0
1,73568.0,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,0
2,73576.0,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,0
3,73577.0,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15,0
4,73580.0,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,0
...,...,...,...,...,...,...,...,...,...,...
2268,83692.0,68.0,1.0,2.0,19.1,89.0,2.0,136.0,1.56,1
2269,83694.0,36.0,2.0,2.0,25.3,99.0,2.0,128.0,20.72,0
2270,83699.0,37.0,2.0,2.0,20.8,83.0,2.0,113.0,1.93,0
2271,83702.0,80.0,2.0,2.0,27.9,111.0,2.0,72.0,13.91,1
